# 7odw

In [1]:
using Printf, DelimitedFiles
using Plots
using DataFrames
using Revise, MDToolbox

In [2]:
function read_ascii(fname)
    lines = readlines(fname)
    natom = length(lines) - 1
    xyz = zeros(Float64, 1, natom*3)
    for i = 1:natom
        xyz[1, 3*(i-1)+1] = parse(Float64, lines[i+1][31:38])
        xyz[1, 3*(i-1)+2] = parse(Float64, lines[i+1][39:46])
        xyz[1, 3*(i-1)+3] = parse(Float64, lines[i+1][47:54])
    end
    TrjArray(xyz=xyz)
end

read_ascii (generic function with 1 method)

In [3]:
fnames = []
omega = []
phi = []
prou = []
d = []
score = []
rmsd = []
struc = []

Any[]

In [4]:
nrun = 1000

1000

In [5]:
for irun = 1:nrun

    if irun % 100 == 0
        print("irun = $irun")
    end
    for i in 1:100
        filename = "$(irun)/complex.$(i).pdb"
        #t = mdload(filename)
        #t = MDToolbox.readpdb_biostructures(filename)
        t = read_ascii(filename)
        t1 = t[:, 1:2026]
        t2 = t[:, 2027:end]
        r = MDToolbox.compute_skrew(t1, t2)
        push!(fnames, filename)
        push!(omega, r.omega)
        push!(phi, r.phi / (2.0 * pi) * 360.0)
        push!(prou, r.prou)
        push!(d, r.d)
        push!(struc, t)
        #t_ca = t["atomname CA"]
        #push!(rmsd, compute_rmsd_permute(ref, t_ca, 2))
    end

    s = readdlm("$(irun)/zdock.out", skipstart=5)
    score = [score; s[1:100, end]]
end

irun = 100irun = 200irun = 300irun = 400irun = 500irun = 600irun = 700irun = 800irun = 900irun = 1000

In [6]:
df = DataFrame(fnames=fnames, score=score, omega=omega, phi=phi, prou=prou, d=d, struc=struc)

Row,fnames,score,omega,phi,prou,d,struc
,Any,Any,Any,Any,Any,Any,Any
1,1/complex.1.pdb,1403.35,"[-0.0912712, 0.710519, 0.697734]",175.461,"[297.126, 27.5862, 10.7756]",12.0772,"1x4052 TrjArray{Float64, Int64}\n| 274.53 207.31 185.82 | … 257.76 173.16 178.07 |"
2,1/complex.2.pdb,1350.98,"[-0.499654, -0.517342, 0.694768]",177.202,"[224.323, 144.834, 269.172]",-1.34399,"1x4052 TrjArray{Float64, Int64}\n| 274.53 207.31 185.82 | … 300.88 247.97 189.68 |"
3,1/complex.3.pdb,1280.75,"[0.70202, -0.202832, -0.682662]",177.052,"[263.81, 222.101, 205.3]",-0.799641,"1x4052 TrjArray{Float64, Int64}\n| 274.53 207.31 185.82 | … 327.50 225.26 170.65 |"
4,1/complex.4.pdb,1276.18,"[-0.699187, 0.423682, -0.575874]",110.304,"[125.816, 283.299, 55.6712]",16.743,"1x4052 TrjArray{Float64, Int64}\n| 274.53 207.31 185.82 | … 239.12 170.38 169.20 |"
5,1/complex.5.pdb,1274.44,"[0.395165, 0.565744, -0.723726]",176.506,"[248.152, 152.542, 254.739]",0.841273,"1x4052 TrjArray{Float64, Int64}\n| 274.53 207.31 185.82 | … 293.94 249.19 188.71 |"
6,1/complex.6.pdb,1261.96,"[-0.435053, 0.713793, 0.548843]",169.03,"[343.223, 123.148, 111.904]",1.45519,"1x4052 TrjArray{Float64, Int64}\n| 274.53 207.31 185.82 | … 310.29 180.03 176.65 |"
7,1/complex.7.pdb,1246.35,"[-0.522153, -0.451985, 0.723233]",157.819,"[230.43, 152.145, 261.447]",-15.6958,"1x4052 TrjArray{Float64, Int64}\n| 274.53 207.31 185.82 | … 323.42 234.92 171.38 |"
8,1/complex.8.pdb,1244.28,"[0.353001, -0.737626, -0.575585]",173.115,"[345.004, 96.0436, 88.5058]",-0.376162,"1x4052 TrjArray{Float64, Int64}\n| 274.53 207.31 185.82 | … 310.07 180.68 170.78 |"
9,1/complex.9.pdb,1234.98,"[-0.420185, -0.488357, 0.764821]",165.077,"[254.711, 162.192, 243.499]",-14.3664,"1x4052 TrjArray{Float64, Int64}\n| 274.53 207.31 185.82 | … 315.57 239.56 168.68 |"


In [7]:
df_org = deepcopy(df);

In [8]:
#using BSON: @save, @load
#@save "skrew_parameters.bson" fnames score omega phi prou d df rmsd

In [9]:
#using BSON: @save, @load
#@save "df.bson" df

In [10]:
using JLD2

save("skrew_parameters.jld2", "fnames", fnames, "score", score, "omega", omega, "phi", phi, "prou", prou, "d", d, "df", df, "rmsd", rmsd)
save("df.jld2", "df", df)